# Stability

In [21]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html, json_item
import json
import IPython

output_file("/home/peter/Documents/cs183/tmp/sinslider.html")


# DEFINE VARIABLES !!!

ELECTRON_CHARGE = 0.1
CG1 = 0.5
CG2 = 0.5
CL = 0.2
CR = 0.2
CM = .1 #1e-5
C1 = CL+CG1+CM
C2 = CR+CG2+CM

EC1 = ELECTRON_CHARGE**2/C1 * (1/(1-CM**2/(C1*C2)))
EC2 = ELECTRON_CHARGE**2/C2 * (1/(1-CM**2/(C1*C2)))
ECM = ELECTRON_CHARGE**2/CM * (1/(C1*C2/CM**2 - 1))

N=4

n_arr = list(range(N ** 2))

v1_type1 = []
v2_type1 = []
v1_type2 = []
v2_type2 = []

for n in n_arr:
    A = n % N
    B = n / N
    # triple point type 1
    v11 = ELECTRON_CHARGE/CG1 * (A + EC2/2 * (EC1-ECM)/(EC1*EC2-ECM**2))
    v1_type1.append(v11)
    v2_type1.append((ELECTRON_CHARGE*((B+.5)*EC2 + A*ECM) - CG1*v11*ECM)/(CG2*EC2))
    # triple point type 2
    v12 = ELECTRON_CHARGE/CG1 * ((A+1) + EC2/2 * (EC1-ECM)/(ECM**2-EC1*EC2))
    v1_type2.append(v12)
    v2_type2.append((ELECTRON_CHARGE*((A+1)*ECM + ((B+1)-.5)*EC2) - CG1*v12*ECM)/(CG2*EC2))

source = ColumnDataSource(data=dict(v1_t1=v1_type1, v2_t1=v2_type1, v1_t2=v1_type2, v2_t2=v2_type2))

plot = figure(y_range=(0, 1), x_range=(0, 1), width=400, height=400)

plot.circle(x='v1_t1', y='v2_t1', color="blue", source=source)
plot.circle(x='v1_t2', y='v2_t2', color="red", source=source)

cm_slider = Slider(start=0.001, end=10, value=1, step=0.1, title='C_M')

callback = CustomJS(args=dict(source=source, ELECTRON_CHARGE=ELECTRON_CHARGE, CG1=CG1, CG2=CG2, CL=CL, CR=CR, N=N, n_arr=n_arr),
                    code="""
    const CM = cb_obj.value

    const C1 = CL+CG1+CM
    const C2 = CR+CG2+CM

    const EC1 = (ELECTRON_CHARGE*ELECTRON_CHARGE)/C1 * (1/(1-CM*CM/(C1*C2)))
    const EC2 = (ELECTRON_CHARGE*ELECTRON_CHARGE)/C2 * (1/(1-CM*CM/(C1*C2)))
    const ECM = (ELECTRON_CHARGE*ELECTRON_CHARGE)/CM * (1/(C1*C2/CM*CM - 1))

    const v1_t1 = Array.from(n_arr, (x) => ELECTRON_CHARGE/CG1 * ((x%N) + EC2/2 * (EC1-ECM)/(EC1*EC2-ECM*ECM)))
    const v2_t1 = Array.from(n_arr, (x) => (ELECTRON_CHARGE*((Math.floor(x/N)+.5)*EC2 + (x%N)*ECM) - CG1*ELECTRON_CHARGE/CG1 * ((x%N) + EC2/2 * (EC1-ECM)/(EC1*EC2-ECM*ECM))*ECM)/(CG2*EC2))
    const v1_t2 = Array.from(n_arr, (x) => ELECTRON_CHARGE/CG1 * (((x%N)+1) + EC2/2 * (EC1-ECM)/(ECM*ECM-EC1*EC2)))
    const v2_t2 = Array.from(n_arr, (x) => (ELECTRON_CHARGE*(((x%N)+1)*ECM + ((Math.floor(x/N)+1)-.5)*EC2) - CG1*ELECTRON_CHARGE/CG1 * (((x%N)+1) + EC2/2 * (EC1-ECM)/(ECM*ECM-EC1*EC2))*ECM)/(CG2*EC2))

    source.data = { v1_t1, v2_t1, v1_t2, v2_t2 }
""")

cm_slider.js_on_change('value', callback)


# item_text = json.dumps(json_item(row(plot, column(amp, freq, phase, offset)), "sinsliders"))
# item = json.parse(item_text)
# embed_item(item);
# html_repr = file_html(row(plot, column(cm_slider)), CDN)
# IPython.display.HTML(html_repr)
show(row(plot, cm_slider))
